In [1]:
#Importar bibliotecas
import pandas as pd
import Bio as bio

In [2]:
#Lectura del archivo con accesiones
df = pd.read_csv("in.txt", sep = '\t')

In [3]:
#Selección de la columna llamada "accesion" y conversión de la columna en lista.
col1 = df["accesion"]
lista = list(col1)
lista

['WIM94906.1', 'QCV87940.1', 'UOE26568.1', 'AIR61364.1']

In [4]:
# FILTRO 1 - TAXONOMÍA
from Bio import SeqIO, Entrez

def extract_protein_data(protein_files):
    # Inicializar listas para almacenar los datos
    protein_data = []
    
    Entrez.email = "scminat@utn.edu.ec"
    Entrez.api_key = "1e71a6fbed3171a40ea7a6cd9e0ed8298009"
    
    for protein_id in protein_files:
        try:
            # Obtener el registro de Protein de la base de datos del NCBI.
            handle = Entrez.efetch(db="protein", id=protein_id, rettype="gb", retmode="text")
            record = SeqIO.read(handle, "genbank")
            handle.close()
            
            # Crear un diccionario para almacenar los datos de cada acceso.
            accession_data = {}
            
            for feature in record.features:
                if feature.type == "CDS":

                    gene_name = feature.qualifiers.get("gene", ["N/A"])[0]
                    
                    # Agregar los datos al diccionario.
                    accession_data["Protein Accession"] = protein_id
            
            # Obtener los datos del taxón.
            taxon_data = record.annotations.get("taxonomy", ["N/A"])
            accession_data["Taxon Data"] = "; ".join(taxon_data)
            
            # Agregar el diccionario a la lista de datos.
            protein_data.append(accession_data)
        
        except Exception as e:
            print(f"Error fetching data for protein ID {protein_id}: {e}")

    # Crear un DataFrame combinado con todos los datos.
    combined_df = pd.DataFrame(protein_data)
    
    # Exportar el DataFrame combinado a un archivo CSV.
    combined_df.to_csv('out.csv', index=False)
    
    # Mostrar el DataFrame combinado.
    print("Combined DataFrame:")
    print(combined_df)
    return combined_df

In [5]:
# Ejemplo de uso
protein_files = lista  # Lista con los IDs de proteínas.
combined_df = extract_protein_data(protein_files)


Combined DataFrame:
  Protein Accession                                         Taxon Data
0        WIM94906.1  Bacteria; Bacillati; Actinomycetota; Actinomyc...
1        QCV87940.1  Bacteria; Bacillati; Actinomycetota; Actinomyc...
2        UOE26568.1  Bacteria; Bacillati; Actinomycetota; Actinomyc...
3        AIR61364.1  Bacteria; Pseudomonadati; Pseudomonadota; Gamm...
